In [1]:
from importlib.metadata import version
print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

import torch
print("torch version:", torch.__version__)


torch version: 2.5.0+cu121
tiktoken version: 0.9.0
torch version: 2.5.0+cu121


In [2]:
import os
import urllib.request

if not os.path.exists("the-verdict.txt"):
    print("Downloading the-verdict.txt...")
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    file_path = "the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)
else:
    print("File already exists")

with open("the-verdict.txt", "r") as file:
    raw_text = file.read()

print("total number of characters:", len(raw_text))
print(raw_text[:10])


File already exists
total number of characters: 20479
I HAD alwa


In [3]:
# 导入正则表达式模块
import re

text = "Hello, world. This, is a test."
# 使用正则表达式分割字符串，保留空白字符
# re.split(pattern, string, maxsplit=0, flags=0)
# pattern: 正则表达式模式
# string: 要分割的字符串
# maxsplit: 最大分割次数，默认为0，表示分割所有匹配项
# flags: 可选参数，用于控制正则表达式的匹配方式（如忽略大小写等）。
# 这里使用r'(\s)'表示匹配所有空白字符，包括空格、制表符、换行符等。
result = re.split(r'(\s)', text)

print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


In [5]:
result = re.split(r'([,.]|\s)', text)

print(result)

# 使用正则表达式分割字符串，不保留空白字符
result = [item.strip() for item in result if item.strip()]
print(result)


['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']
['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


In [6]:
# 处理其他特殊符号
text = "Hello, world. Is this-- a test?"
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item for item in result if item.strip()]
print(result)


['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


In [8]:
preprocessed_text = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)

preprocessed_text = [item.strip() for item in preprocessed_text]
print(preprocessed_text[:30])
print(len(preprocessed_text))

preprocessed_text = [item.strip() for item in preprocessed_text if item.strip()]

print(preprocessed_text[:30])
print(len(preprocessed_text))





['I', '', 'HAD', '', 'always', '', 'thought', '', 'Jack', '', 'Gisburn', '', 'rather', '', 'a', '', 'cheap', '', 'genius', '--', 'though', '', 'a', '', 'good', '', 'fellow', '', 'enough', '--']
9235
['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']
4690


In [9]:
all_words = sorted(set(preprocessed_text))
vocabulary_size = len(all_words)

print(f"Vocabulary size: {vocabulary_size}")
vocabulary = {token:index for index, token in enumerate(all_words)}
for i, item in enumerate(vocabulary.items()):
    print(f"{i}: {item}")
    if i >= 10:
        break



Vocabulary size: 1130
0: ('!', 0)
1: ('"', 1)
2: ("'", 2)
3: ('(', 3)
4: (')', 4)
5: (',', 5)
6: ('--', 6)
7: ('.', 7)
8: (':', 8)
9: (';', 9)
10: ('?', 10)


In [10]:
class SimpleTokenizerV1:
    def __init__(self, vocabulary):
        # vocabulary 是字典，键是字符串，值是整数(token, index)
        self.str_to_int = vocabulary
        # int_to_str 是字典，键是整数，值是字符串(index, token)
        self.int_to_str = {i:s for s, i in vocabulary.items()}
    
    def encode(self, text):
        # 将文本分割为单词
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        # 去除空字符串
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        # 将单词转换为整数
        idx = [self.str_to_int[item] for item in preprocessed]
        return idx
    
    def decode(self, idx):
        text = " ".join([self.int_to_str[i] for i in idx])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text
    

tokenizer = SimpleTokenizerV1(vocabulary)

text = """"It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""
idx = tokenizer.encode(text)
print(idx)
print(tokenizer.decode(idx))
print(tokenizer.decode(tokenizer.encode(text)))

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]
" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.
" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


In [11]:
tokenizer = SimpleTokenizerV1(vocabulary)

text = "Hello, do you like tea. Is this-- a test?"

tokenizer.encode(text)

KeyError: 'Hello'

In [12]:
print(len(vocabulary))

all_tokens = sorted(set(preprocessed_text))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocabulary = {token:index for index, token in enumerate(all_tokens)}

print(len(vocabulary))

for i, item in enumerate(list(vocabulary.items())[-5:]):
    print(f"{i}: {item}")



1130
1132
0: ('younger', 1127)
1: ('your', 1128)
2: ('yourself', 1129)
3: ('<|endoftext|>', 1130)
4: ('<|unk|>', 1131)


In [13]:
class SimpleTokenizerV2:
    def __init__(self, vocabulary):
        self.str_to_int = vocabulary
        self.int_to_str = {i:s for s, i in vocabulary.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int else "<|unk|>"
            for item in preprocessed
        ]

        idx = [self.str_to_int[item] for item in preprocessed]
        return idx
    
    def decode(self, idx):
        text = " ".join([self.int_to_str[i] for i in idx])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

tokenizer = SimpleTokenizerV2(vocabulary)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join((text1, text2))

print(text)
print(tokenizer.encode(text))
print(tokenizer.decode(tokenizer.encode(text)))



Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.
[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]
<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.


In [14]:
import importlib
import tiktoken

print("tiktoken version:", tiktoken.__version__)
print("tiktoken version:", importlib.metadata.version("tiktoken"))

tokenizer = tiktoken.get_encoding("gpt2")

text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)
string = tokenizer.decode(integers)
print(string)
print(text)
print(text == string)


tiktoken version: 0.9.0
tiktoken version: 0.9.0
[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]
Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.
Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.
True


In [18]:
with open("the-verdict.txt", "r", encoding="utf-8") as file:
    raw_text = file.read()

encoded_text = tokenizer.encode(raw_text)
print(len(encoded_text))

encoded_sample = encoded_text[50:]
context_size = 4

x = [encoded_sample[i:i+context_size] for i in range(0, len(encoded_sample))]
for i in range(len(x)):
    if i >= 10:
        break
    print(f"x[{i}]: {x[i]}")

x = encoded_sample[:context_size]
y = encoded_sample[1:context_size+1]

print(f"x: {x}")
print(f"y:      {y}")


5145
x[0]: [290, 4920, 2241, 287]
x[1]: [4920, 2241, 287, 257]
x[2]: [2241, 287, 257, 4489]
x[3]: [287, 257, 4489, 64]
x[4]: [257, 4489, 64, 319]
x[5]: [4489, 64, 319, 262]
x[6]: [64, 319, 262, 34686]
x[7]: [319, 262, 34686, 41976]
x[8]: [262, 34686, 41976, 13]
x[9]: [34686, 41976, 13, 357]
x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


In [19]:
for i in range(1, 10):
    context = encoded_sample[:i]
    desired = encoded_sample[i]
    print(context, "---->", desired)
    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))


[290] ----> 4920
 and ---->  established
[290, 4920] ----> 2241
 and established ---->  himself
[290, 4920, 2241] ----> 287
 and established himself ---->  in
[290, 4920, 2241, 287] ----> 257
 and established himself in ---->  a
[290, 4920, 2241, 287, 257] ----> 4489
 and established himself in a ---->  vill
[290, 4920, 2241, 287, 257, 4489] ----> 64
 and established himself in a vill ----> a
[290, 4920, 2241, 287, 257, 4489, 64] ----> 319
 and established himself in a villa ---->  on
[290, 4920, 2241, 287, 257, 4489, 64, 319] ----> 262
 and established himself in a villa on ---->  the
[290, 4920, 2241, 287, 257, 4489, 64, 319, 262] ----> 34686
 and established himself in a villa on the ---->  Riv


In [26]:
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, text, tokenizer, max_length, stride):
        self.input_idx = []
        self.target_idx = []

        # tokenize the entire text
        token_idx = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

        # use a sliding window to chunk the book into overlapping sequences
        # 将token_idx分割为max_length大小的序列
        # 将chunk加入到input_idx和target_idx
        for i in range(0, len(token_idx) - max_length, stride):
            input_chunk = token_idx[i: i + max_length]
            target_chunk = token_idx[i + 1: i + max_length + 1]

            if i == 0:
                print(len(input_chunk), len(target_chunk))
                print(input_chunk)
                print(target_chunk)
                print(torch.tensor(input_chunk))
                print(torch.tensor(target_chunk))
            self.input_idx.append(torch.tensor(input_chunk))
            self.target_idx.append(torch.tensor(target_chunk))
    
    def __len__(self):
        return len(self.input_idx)

    def __getitem__(self, idx):
        return self.input_idx[idx], self.target_idx[idx]


In [29]:
def create_dataloader_v1(text, batch_size=4, max_length=256,
                        stride=128, shuffle=True, drop_last=True,
                        num_workers=0):

    # initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # create the dataset
    dataset = GPTDatasetV1(text, tokenizer, max_length, stride)

    # create the dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )
    return dataloader

with open("the-verdict.txt", "r", encoding="utf-8") as file:
    raw_text = file.read()


dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)

print(len(dataloader))
data_iter = iter(dataloader)
first_batch = next(data_iter)
second_batch = next(data_iter)

print(first_batch)
print(second_batch)

inputs = first_batch[0]
targets = first_batch[1]

print(inputs.shape)
print(targets.shape)

print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

inputs = second_batch[0]
targets = second_batch[1]

print(inputs.shape)
print(targets.shape)

print("Inputs:\n", inputs)
print("\nTargets:\n", targets)


4 4
[40, 367, 2885, 1464]
[367, 2885, 1464, 1807]
tensor([  40,  367, 2885, 1464])
tensor([ 367, 2885, 1464, 1807])
5141
[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]
[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]
torch.Size([1, 4])
torch.Size([1, 4])
Inputs:
 tensor([[  40,  367, 2885, 1464]])

Targets:
 tensor([[ 367, 2885, 1464, 1807]])
torch.Size([1, 4])
torch.Size([1, 4])
Inputs:
 tensor([[ 367, 2885, 1464, 1807]])

Targets:
 tensor([[2885, 1464, 1807, 3619]])


In [30]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)

print(inputs.shape)
print(targets.shape)

print("Inputs:\n", inputs)
print("\nTargets:\n", targets)



4 4
[40, 367, 2885, 1464]
[367, 2885, 1464, 1807]
tensor([  40,  367, 2885, 1464])
tensor([ 367, 2885, 1464, 1807])
torch.Size([8, 4])
torch.Size([8, 4])
Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


In [31]:
input_idx = torch.tensor([2, 3, 5, 1])

vocabulary_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocabulary_size, output_dim)

print(embedding_layer.weight)
print(embedding_layer(torch.tensor([3])))
print(embedding_layer(input_idx))



Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)
tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)
tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


In [32]:
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [33]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False
)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)

print(inputs.shape)
print(targets.shape)

print(inputs)



4 4
[40, 367, 2885, 1464]
[367, 2885, 1464, 1807]
tensor([  40,  367, 2885, 1464])
tensor([ 367, 2885, 1464, 1807])
torch.Size([8, 4])
torch.Size([8, 4])
tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])


In [34]:
token_embeddings = token_embedding_layer(inputs)

print(token_embeddings.shape)

print(token_embeddings)


torch.Size([8, 4, 256])
tensor([[[ 0.4913,  1.1239,  1.4588,  ..., -0.3995, -1.8735, -0.1445],
         [ 0.4481,  0.2536, -0.2655,  ...,  0.4997, -1.1991, -1.1844],
         [-0.2507, -0.0546,  0.6687,  ...,  0.9618,  2.3737, -0.0528],
         [ 0.9457,  0.8657,  1.6191,  ..., -0.4544, -0.7460,  0.3483]],

        [[ 1.5460,  1.7368, -0.7848,  ..., -0.1004,  0.8584, -0.3421],
         [-1.8622, -0.1914, -0.3812,  ...,  1.1220, -0.3496,  0.6091],
         [ 1.9847, -0.6483, -0.1415,  ..., -0.3841, -0.9355,  1.4478],
         [ 0.9647,  1.2974, -1.6207,  ...,  1.1463,  1.5797,  0.3969]],

        [[-0.7713,  0.6572,  0.1663,  ..., -0.8044,  0.0542,  0.7426],
         [ 0.8046,  0.5047,  1.2922,  ...,  1.4648,  0.4097,  0.3205],
         [ 0.0795, -1.7636,  0.5750,  ...,  2.1823,  1.8231, -0.3635],
         [ 0.4267, -0.0647,  0.5686,  ..., -0.5209,  1.3065,  0.8473]],

        ...,

        [[-1.6156,  0.9610, -2.6437,  ..., -0.9645,  1.0888,  1.6383],
         [-0.3985, -0.9235, -1.31

In [38]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

print(pos_embedding_layer.weight)
print(pos_embedding_layer.weight.shape)
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print(pos_embeddings.shape)

input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)



Parameter containing:
tensor([[ 1.9192, -0.0239, -1.3632,  ...,  1.0244,  0.6319, -0.2026],
        [ 1.0580, -0.1855,  0.5553,  ...,  0.0593, -1.2205,  0.7917],
        [ 0.3932,  0.1786, -0.4253,  ...,  1.4493, -0.8674,  0.8658],
        [ 0.4966, -1.9540, -1.0235,  ...,  1.1835,  1.0848,  0.1064]],
       requires_grad=True)
torch.Size([4, 256])
torch.Size([4, 256])
torch.Size([8, 4, 256])
